In [12]:
from scipy.interpolate import RegularGridInterpolator
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# =============================================================================
# 4. Load FEM Data and Build Interpolation Functions
# =============================================================================
dir_path = ".."
FEM_dataset = "100x100mm.dat"
L_max = 100.0

fem_file = os.path.join(dir_path, r"data_fem", FEM_dataset)
data = np.loadtxt(fem_file)
X_val      = data[:, :2]
u_val      = data[:, 2:4]
strain_val = data[:, 4:7]
stress_val = data[:, 7:10]
solution_val = np.hstack((u_val, stress_val))

n_mesh_points = int(np.sqrt(X_val.shape[0]))
x_grid = np.linspace(0, L_max, n_mesh_points)
y_grid = np.linspace(0, L_max, n_mesh_points)

def create_interpolation_fn(data_array):
    num_components = data_array.shape[1]
    interpolators = []
    for i in range(num_components):
        interp = RegularGridInterpolator(
            (x_grid, y_grid),
            data_array[:, i].reshape(n_mesh_points, n_mesh_points).T,
        )
        interpolators.append(interp)
    def interpolation_fn(x):
        return np.array([interp((x[:, 0], x[:, 1])) for interp in interpolators]).T
    return interpolation_fn

solution_fn = create_interpolation_fn(solution_val)
strain_fn   = create_interpolation_fn(strain_val)

In [57]:
from scipy.spatial import cKDTree

def load_data(
    DIC_dataset_path,
    DIC_dataset_number,
    strain_fn,
    num_measurments,
    noise_magnitude,
    DIC_region,
    pad_extrapolate=False,
    dir_path='..',
    verbose=True,
):
    """
    Load DIC data (or synthetic), subsample uniformly over full_region to
    num_measurments^2 points, then apply DIC_region mask.
    If pad_extrapolate, out-of-region strains are replaced by nearest in-region.
    Prints grid sizes as Full: n_f x n_f --> Region: n_r1 x n_r2 (or keeps Full with padding).
    """
    if DIC_dataset_path != 'no_dataset':
        base = os.path.join(dir_path, DIC_dataset_path)
        n = DIC_dataset_number
        Xg = pd.read_csv(os.path.join(base, 'x', f'x_{n}.csv'), delimiter=';').dropna(axis=1).to_numpy()
        Yg = pd.read_csv(os.path.join(base, 'y', f'y_{n}.csv'), delimiter=';').dropna(axis=1).to_numpy()
        Exx = pd.read_csv(os.path.join(base, 'exx', f'exx_{n}.csv'), delimiter=';').dropna(axis=1).to_numpy()
        Eyy = pd.read_csv(os.path.join(base, 'eyy', f'eyy_{n}.csv'), delimiter=';').dropna(axis=1).to_numpy()
        Exy = pd.read_csv(os.path.join(base, 'exy', f'exy_{n}.csv'), delimiter=';').dropna(axis=1).to_numpy()
        coords, data = postprocess_dic_data(
            Xg, Yg, Exx, Eyy, Exy,
            DIC_region=DIC_region,
            num_measurments=num_measurments,
            pad_extrapolate=pad_extrapolate,
            verbose=verbose,
        )
    else:
        xmin, ymin, xmax, ymax = DIC_region
        xs = np.linspace(xmin, xmax, num_measurments)
        ys = np.linspace(ymin, ymax, num_measurments)
        Xg, Yg = np.meshgrid(xs, ys)
        coords = np.column_stack([Xg.ravel(), Yg.ravel()])
        data = strain_fn(coords)
        data += np.random.normal(0, noise_magnitude, data.shape)
        if verbose:
            print(f"Dic grid: {num_measurments} x {num_measurments} with {noise_magnitude} magnitude noise")
    return coords, data


def postprocess_dic_data(
    Xg, Yg, Exx, Eyy, Exy,
    DIC_region,
    num_measurments,
    pad_extrapolate=False,
    verbose=True,
):
    # Subsample full grid uniformly by index
    m, n = Xg.shape
    i_full = np.round(np.linspace(0, m-1, min(m, num_measurments))).astype(int)
    j_full = np.round(np.linspace(0, n-1, min(n, num_measurments))).astype(int)
    n_f = i_full.size
    # Extract subsampled coords and values
    sel_full = np.ix_(i_full, j_full)
    Xf = Xg[sel_full]; Yf = Yg[sel_full]
    Exx_f = Exx[sel_full]; Eyy_f = Eyy[sel_full]; Exy_f = Exy[sel_full]
    # Region mask on subsampled grid
    xmin, ymin, xmax, ymax = DIC_region
    mask = (Xf>=xmin)&(Xf<=xmax)&(Yf>=ymin)&(Yf<=ymax)
    # Count region rows/cols
    rows_r, cols_r = np.unique(np.where(mask)[0]), np.unique(np.where(mask)[1])
    # Print sizes
    if verbose:
        if pad_extrapolate:
            print(f"DIC grid: {m} x {n} --> Subsampled grid (including padding): {n_f} x {n_f} --> Region grid: {rows_r.size} x {cols_r.size}")
        else:
            print(f"DIC grid: {m} x {n} --> Subsampled grid: {n_f} x {n_f} --> Region grid (points used): {rows_r.size} x {cols_r.size}")
    # Apply mask or padding
    if pad_extrapolate:
        # build in-region tree
        pts_in = np.column_stack([Xf[mask].ravel(), Yf[mask].ravel()])
        vals_in = np.column_stack([Exx_f[mask].ravel(), Eyy_f[mask].ravel(), Exy_f[mask].ravel()])
        tree = cKDTree(pts_in)
        pts_all = np.column_stack([Xf.ravel(), Yf.ravel()])
        _, idx = tree.query(pts_all)
        filled = vals_in[idx]
        coords = pts_all
        data = filled
    else:
        coords = np.column_stack([Xf[mask].ravel(), Yf[mask].ravel()])
        data = np.column_stack([Exx_f[mask].ravel(), Eyy_f[mask].ravel(), Exy_f[mask].ravel()])
    return coords, data


In [58]:
dir_path = ".."

# --- 2) Problem parameters from your FEniCS code ---
L = 100.0   # mm
m = 0.3     # N/mm (side‐loading slope)
b = 50.0    # N (side‐loading intercept)

# --- 3) Compute total shear load on the right boundary ---
#    ∫[0→L] (m y + b) dy = m L^2/2 + b L
F = m*L**2/2 + b*L

def calc_parameters(Eps1, Eps2, Eps6):
        """
        Calculate the E and ν parameters from the strain data.
        """
        # build A and B
        A = np.zeros((2, 2))
        B = np.zeros(2)

        # Virtual field 1:  u1*=0, u2*=-x2  ⇒ ε1*=0, ε2*=-1
        A[0, 0] = np.mean(Eps2)*L**2     
        A[0, 1] = np.mean(Eps1)*L**2      
        B[0]    = 0        

        # Virtual field 2:  u1*=x1, u2*=0  ⇒ ε1*=1, ε2*=0
        A[1, 0] = np.mean(Eps1)*L**2       # ∑ ε₁ sᵢ
        A[1, 1] = np.mean(Eps2)*L**2      # ∑ ε₂ sᵢ
        B[1]    = F * L                # ∑ σ₁ lᵢ

        # solve for Q = [Q11, Q12]
        Q11, Q12 = np.linalg.solve(A, B)

        # retrieve E and ν
        μ_id = (Q11 - Q12)/2
        λ_id = Q12
        Nu  = λ_id / (2*(μ_id + λ_id))
        E  = μ_id*(3*λ_id + 2*μ_id)/(λ_id + μ_id)
        return E, Nu

prop = 0
DIC_region = [prop*L, prop*L, (1-prop)*L, (1-prop)*L]

E_list = []
Nu_list = []

for i in range(1, 20):
        X_DIC_input, DIC_data = load_data("no_dataset", 0, strain_fn, 4, 1e-6, DIC_region=DIC_region, verbose=(i==1))
        Eps1 = DIC_data[:, 0].reshape(-1, 1)
        Eps2 = DIC_data[:, 1].reshape(-1, 1)
        Eps6 = DIC_data[:, 2].reshape(-1, 1)

        E, Nu = calc_parameters(Eps1, Eps2, Eps6)
        E_list.append(E)
        Nu_list.append(Nu)

E_list = np.array(E_list)
Nu_list = np.array(Nu_list)
print(f"Identified:  E = {np.mean(E_list)/1e3:.4f} ± {np.std(E_list)/1e3:.4f} GPa, ν = {np.mean(Nu_list):.4f} ± {np.std(Nu_list):.4f}")

Dic grid: 4 x 4 with 1e-06 magnitude noise
Identified:  E = 207.0592 ± 0.1316 GPa, ν = 0.3051 ± 0.0005


In [62]:
camera_resolution = "2MP"
DIC_dataset_path = f"2_noise_study/data_dic/{camera_resolution}/1noise"
prop = 0.1
DIC_region = [prop*L, prop*L, (1-prop)*L, (1-prop)*L]
# pad_extrapolate = True
num_measurments = 6

for pad_extrapolate in [True, False]:
    # num_measurments = 6 if pad_extrapolate else 10
    E_list = []
    Nu_list = []
    for dic_number in range(1, 11):
        X_DIC_input, DIC_data = load_data(DIC_dataset_path, dic_number, strain_fn, num_measurments, 1e-6, DIC_region=DIC_region, pad_extrapolate=pad_extrapolate, verbose=(dic_number==1))

        Eps1 = DIC_data[:, 0].reshape(-1, 1)
        Eps2 = DIC_data[:, 1].reshape(-1, 1)
        Eps6 = DIC_data[:, 2].reshape(-1, 1)

        E, Nu = calc_parameters(Eps1, Eps2, Eps6)
        E_list.append(E)
        Nu_list.append(Nu)

    E_list = np.array(E_list)
    Nu_list = np.array(Nu_list)
    padding = "Padding:" if pad_extrapolate else "No padding:"

    print(f"{padding}  E = {np.mean(E_list)/1e3:.4f} ± {np.std(E_list)/1e3:.4f} GPa,   ν = {np.mean(Nu_list):.4f} ± {np.std(Nu_list):.4f}")

DIC grid: 114 x 115 --> Subsampled grid (including padding): 6 x 6 --> Region grid: 4 x 4
Padding:  E = 211.3346 ± 0.1827 GPa,   ν = 0.2978 ± 0.0004
DIC grid: 114 x 115 --> Subsampled grid: 6 x 6 --> Region grid (points used): 4 x 4
No padding:  E = 211.7390 ± 0.1643 GPa,   ν = 0.2971 ± 0.0003


In [65]:
import json
import numpy as np

# Assuming load_data and calc_parameters are defined elsewhere
# Define your variables and functions here

resolutions = ["5MP", "2MP", "0.4MP"]
prop = 0
E_actual = 210.0 # Actual Young's modulus in GPa
Nu_actual = 0.3 # Actual Poisson's ratio
num_measurments = 1000

# Load the existing JSON file
with open("../2_noise_study/results/summary.json", 'r') as json_file:
    json_data = json.load(json_file)

for camera_resolution in resolutions:
    DIC_dataset_path = f"2_noise_study/data_dic/{camera_resolution}/1noise"
    DIC_region = [prop*L, prop*L, (1-prop)*L, (1-prop)*L]

    E_list = []
    Nu_list = []
    for dic_number in range(1, 11):
        X_DIC_input, DIC_data = load_data(DIC_dataset_path, dic_number, strain_fn, num_measurments, 1e-6, DIC_region=DIC_region, verbose=(dic_number==1))
        Eps1 = DIC_data[:, 0].reshape(-1, 1)
        Eps2 = DIC_data[:, 1].reshape(-1, 1)
        Eps6 = DIC_data[:, 2].reshape(-1, 1)

        E, Nu = calc_parameters(Eps1, Eps2, Eps6)
        E_list.append(E)
        Nu_list.append(Nu)

    E_list = np.array(E_list)/1e3  # Convert to GPa
    Nu_list = np.array(Nu_list)

    json_data[camera_resolution]["VFM"] = {}

    json_data[camera_resolution]["VFM"]["E"] = {"mean": np.mean(E_list), "std": np.std(E_list), "mean_rel_error": np.mean(np.abs((E_list - E_actual) / E_actual))*100, "std_rel_error": np.std(np.abs((E_list - E_actual) / E_actual))*100}
    json_data[camera_resolution]["VFM"]["nu"] = {"mean": np.mean(Nu_list), "std": np.std(Nu_list), "mean_rel_error": np.mean(np.abs((Nu_list - Nu_actual) / Nu_actual))*100, "std_rel_error": np.std(np.abs((Nu_list - Nu_actual) / Nu_actual))*100}

# Save the updated JSON structure back to the file
with open("../2_noise_study/results/summary.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=4)


DIC grid: 191 x 192 --> Subsampled grid: 191 x 191 --> Region grid (points used): 191 x 192
DIC grid: 114 x 115 --> Subsampled grid: 114 x 114 --> Region grid (points used): 114 x 115
DIC grid: 39 x 41 --> Subsampled grid: 39 x 39 --> Region grid (points used): 39 x 41
